# []

In [ ]:
# inbuilt 
import os
import sys
import math

# most common
import numpy as np
import matplotlib.pyplot as plt

# pytorch
import torch as tt
import torch.nn as nn
import torch.optim as oo
import torch.functional as ff
import torch.distributions as dd
import torch.utils.data as ud

# custom
import known

print(f'{sys.version=}\n{np.__version__=}\n{tt.__version__=}\n{known.__version__=}')

In [ ]:
import known.ktorch as kt

In [ ]:
seqlen = 10
ds = kt.SeqDataset.from_csv(csv='PJDS.csv', col='PRICE', reverse=True, seqlen=seqlen, dtype=tt.float32  )
ds

In [ ]:


class RnnMlp(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.rnn = kt.StackedLSTM(
            input_size=1,
            hidden_sizes=(20,20,20),
            nlF=nn.Tanh, nlF2=nn.Tanh, bias=False, dropout=0.0, dtype=tt.float32, batch_first=True
        )
        self.fc = nn.Sequential( nn.Flatten(), nn.Linear(200, 1))
    
    def forward(self, X):
        x, _, _ = self.rnn(X)
        y = self.fc(x)
        return y


In [ ]:
rnm = RnnMlp()

In [ ]:
history = kt.Trainer.train( rnm,
    training_data=ds, 
    validation_data=ds, 
    epochs=200, 
    batch_size=32, 
    shuffle=True, 
    validation_freq=5, 
    criterion_type=nn.MSELoss, 
    criterion_args={}, 
    optimizer_type=oo.Adam, 
    optimizer_args={'lr': 0.00005, 'weight_decay': 0.0}, 
    lrs_type=oo.lr_scheduler.LinearLR, 
    lrs_args={'start_factor': 1.0, 'end_factor':0.7, 'total_iters': 50},
    record_batch_loss=False, 
    early_stop_train=kt.QuantiyMonitor('TrainLoss', patience=6, delta=0.01, verbose=True), 
    early_stop_val=kt.QuantiyMonitor('ValLoss', patience=6, delta=0.01, verbose=True), 
    checkpoint_freq=5, 
    save_path='sample.rnn',
    save_state_only=True, 
    verbose=2, 
    plot=1
)